# Applications Weighted Grading Automation

(*By: [@mahmoud-elmakki](https://github.com/mahmoud-elmakki)*)

The main objecctive of this code is to automate the process of calculating the overall weighted grade of students' responses, from the weights and grades given to each question from the graded questions.

In [11]:
import numpy as np
import pandas as pd

import os

In [12]:
# Note that you have to download the responses data Excel sheet from Google Drive and put it in the same folder as the code.
# You don't have to do this if you cloned the Github repo (all will be organized in the repo).
 
# TODO: Load data directly from Google Drive.

# Loading students responses data
GRD_STDN_DATA_DIR = './responses_with_graded_answers/Graded TReND Comp Neuro application form Rwanda 2024 (Responses).xlsx'
grd_stdn_df = pd.read_excel(GRD_STDN_DATA_DIR)

grd_stdn_df['Total Grade'] = None

# Just specify folder names - thje code will create the directory
OUTPUT_FOLDER_NAME = "final_output"
OUTPUT_DIR = os.path.join(os.getcwd(), OUTPUT_FOLDER_NAME)

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [13]:
# Use this dictionary as a reference for column names.

qs_dict = {i: column for i, column in enumerate(grd_stdn_df.columns)}
qs_dict

{0: 'Timestamp',
 1: 'Email address',
 2: 'First Name',
 3: 'Last Name',
 4: 'Age',
 5: 'Nationality',
 6: 'In case you are selected, where would you be traveling from to Rwanda?',
 7: 'Gender',
 8: 'What is your highest degree of education?',
 9: 'What is your current career stage?',
 10: 'Which group of studies best describes your background?',
 11: 'Name of current University/Research Institution/Organization (e.g. Department of Biomedical Engineering, CMU-Africa, Kigali, Rwanda)',
 12: 'Field of studies in Undergraduate (completed or ongoing, eg. Neuroscience, Mathematics, Law)',
 13: 'Field of studies in Master’s (if applicable, completed or ongoing, eg. Neuroscience, Mathematics, Law)',
 14: 'Field of studies in PhD/MD degree (if applicable, completed or ongoing, eg. Neuroscience, Mathematics, Law)',
 15: 'Current research focus or research focus of the last research project you were engaged in (if applicable)',
 16: 'Rate your proficiency of computer programming (with any progra

In [14]:
# Used indices of the student responses DataFrame

stdn_idcs = {
    'email_idx' : 1,
    'firstname_idx' : 2,
    'lastname_idx' : 3,
    'ref' : {
        'first_ref_email_idx' : 25,
        'second_ref_email_idx' : 27
          },
    'flag_idx' : 28,
    'notes_idx' : 29,
    'first_recomm_letter_idx' : 30,
    'second_recomm_letter_idx' : 31,
    'grade_idx': 28
}

stdn_str_qs = [stdn_idcs['email_idx'], stdn_idcs['firstname_idx'], stdn_idcs['lastname_idx']]

# Carefully specify names of the columns to be processed (mostly responses for essay questions).
skills_qs = [16, 17, 18, 19]
skills_qs_ws = [1, 2, 2, 2]

# Carefully specify names of the columns to be processed (mostly responses for essay questions).
essay_qs = [20, 21, 22]
essay_qs_ws = [3, 3, 2]

grd_qs = skills_qs + essay_qs
weights = skills_qs_ws + essay_qs_ws

WEIGHTED_GRADIN = True

if not WEIGHTED_GRADIN:
    weights = [1, 1, 1, 1, 1, 1, 1]

# Number of students to be selected.
TOP_N = 20

if TOP_N < len(grd_stdn_df):
    TOP_N = len(grd_stdn_df)

In [15]:
skills_qs_dict = {
    
    16: {
        'Have not done any computer programming': 2,
        'Learned some, edited scripts': 4,
        'Have written my own sets of code': 6,
        'Have written my own complete programs': 8,
        'I am completely comfortable writing my own programs': 10
        
    },
    
    17: {
        'Have not touched Python yet': 2,
        'Learned some, edited scripts': 4,
        'Have written my own sets of code': 6,
        'Have written my own complete programs': 8,
        'I am completely comfortable writing my own programs': 10
    },
    
    18: {
        'Have read a bit': 2,
        'Have taken an introductory course': 4,
        'Have taken several courses': 6,
        'Have taken courses and am studying it': 8,
        'Extensive experience for 4+ years': 10
    },
    
    19: {
        'Have read a bit': 2,
        'Have taken an introductory course': 4,
        'Have taken several courses': 6,
        'Have taken courses and am studying it': 8,
        'Extensive experience for 4+ years': 10
    }

}

In [16]:
def column_names_to_indices(df, indices_dict):
    """
    Replaces column names with indices.
    """
    processed_df = df.rename(columns={column: i for i, column in enumerate(indices_dict.values())})

    return processed_df


def indices_to_column_names(df, indices_dict):
    """
    Replaces indices with column names.
    """
    processed_df = df.rename(columns={i: column for i, column in enumerate(indices_dict.values())})

    return processed_df


def calc_grade(grd_stdn_df):
    
    for row_idx in range(len(grd_stdn_df)):
        grds = []
        
        for q in skills_qs:
            grd = float(skills_qs_dict[q][grd_stdn_df.iloc[row_idx, q]])
            grds.append(grd)
            
        for q in essay_qs:
            grd = float(grd_stdn_df.iloc[row_idx, q])
            grds.append(grd)
        
        weighted_grds = [grds[i] * weights[i] for i in range(len(grds))]
        total_grd = float(round(sum(weighted_grds) / sum(weights), 2))
        grd_stdn_df.iloc[row_idx, stdn_idcs['grade_idx']] = total_grd
            
    return grd_stdn_df
 

In [17]:
def main(grd_stdn_df):
    
    grd_stdn_df = column_names_to_indices(grd_stdn_df, qs_dict)
    grd_stdn_df_graded = calc_grade(grd_stdn_df)
    grd_stdn_df_sorted = grd_stdn_df_graded.sort_values(by = stdn_idcs['grade_idx'], ascending = False)
    
    grd_stdn_df_named = indices_to_column_names(grd_stdn_df_sorted, qs_dict)
    grd_stdn_df_named.to_excel(OUTPUT_FOLDER_NAME + "/graded_responses.xlsx")
    
    top_n_stnds = grd_stdn_df_sorted.iloc[:TOP_N, :]
    
    top_n_stnds_nammed = indices_to_column_names(grd_stdn_df_graded, qs_dict)
    top_n_stnds_nammed.to_excel(OUTPUT_FOLDER_NAME + "/top_n_graded_responses.xlsx")
    
    return grd_stdn_df_sorted, top_n_stnds

In [18]:
grd_stdn_df_graded, top_n_stnds = main(grd_stdn_df)

In [19]:
grd_stdn_df_graded.iloc[:, stdn_idcs['grade_idx']]

17     9.0
36    8.47
34    8.07
19    7.07
3      7.0
0     6.93
29     6.8
9     6.67
35     6.6
18    6.53
20    6.47
7     6.33
1     5.93
16     5.8
15    5.67
8      5.6
2     5.53
33    5.53
21    5.53
28    5.47
26     5.4
5      5.4
10    5.33
30    5.33
31    5.13
6     5.13
11    5.07
22    4.73
27    4.67
13    4.67
24     4.6
23     4.6
14     4.6
32    4.53
12    4.33
25     3.8
4     3.53
Name: 28, dtype: object

In [20]:
top_n_stnds.iloc[:, stdn_idcs['grade_idx']]

17     9.0
36    8.47
34    8.07
19    7.07
3      7.0
0     6.93
29     6.8
9     6.67
35     6.6
18    6.53
20    6.47
7     6.33
1     5.93
16     5.8
15    5.67
8      5.6
2     5.53
33    5.53
21    5.53
28    5.47
26     5.4
5      5.4
10    5.33
30    5.33
31    5.13
6     5.13
11    5.07
22    4.73
27    4.67
13    4.67
24     4.6
23     4.6
14     4.6
32    4.53
12    4.33
25     3.8
4     3.53
Name: 28, dtype: object

In [21]:
top_n_stnds

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
17,2024-01-17 12:10:54.634,yahigbm@gmail.com,Youcef,YAHI,29,Algeria,Algeria,Male,Master's,PhD,...,Extensive experience for 4+ years,8,9,7,https://drive.google.com/open?id=1dRbm4_xxkPBp...,NaN,NaN,NaN,NaN,9.0
36,2024-01-19 08:26:05.172,sniyizurugero@aimsric.org,Samson,NIYIZURUGERO,27,Rwanda,Rwanda,Male,Master's,Master student,...,Have taken courses and am studying it,8,9,7,https://drive.google.com/open?id=1sHbLC_TarkTb...,Dr. Paterne Gahungu/African Institute for Math...,paterne.gahungu@aims.ac.rw,Blaise Tchapnda/African Institute for Mathemat...,blaise.tchapnda@aims.ac.rw,8.47
34,2024-01-18 17:44:47.464,Sbganawakili@gmail.com,Shuaibu,Babagana,33,Nigeria,Nigeria,Male,Bachelor's degree,Master student,...,Have taken courses and am studying it,7,10,4,https://drive.google.com/open?id=1vqZYfH56JV_4...,Assistant Professor Kaloma Usman Majikumna/Uni...,kalomausman@unimaid.edu.ng,"Dr. Mohammed Babakura/Borno State University, ...",mohamedbabakura@bosu.edu.ng,8.07
19,2024-01-17 12:37:42.406,eakanko15@gmail.com,Erica,Akanko,25,Ghana,Ghana,Female,Bachelor's degree,Undergraduate,...,Have taken several courses,8,6,6,https://drive.google.com/open?id=13DuCK0nWozKk...,Prof Michael D. Wilson / Noguchi Memorial Inst...,MWilson@noguchi.ug.edu.gh,Mathew Abrams / International Neuroinformatics...,mathew.abrams@incf.org,7.07
3,2024-01-16 15:23:31.832,2567491@students.wits.ac.za,Talha,Niazi,20,South Africa,South Africa,Male,High school degree,Undergraduate,...,Have taken courses and am studying it,1,8,8,https://drive.google.com/open?id=1T_1wA5oZllp1...,Vanitha Rajamany/Redhill School,vrajamany@redhill.co.za,Nick Smit/Smergos,nick@smergos.com,7.0
0,2024-01-16 08:06:27.194,jakindaodhiambo@gmail.com,Jakinda,Oluoch,25,Kenya,Kenya,Male,Bachelor's degree,Working in industry,...,Have taken courses and am studying it,8,6,6,https://drive.google.com/open?id=1q_WYgQLQHGLO...,Benard Alaka,\tbalaka@strathmore.edu,Wallace Muchiri,wmuchiri@strathmore.edu,6.93
29,2024-01-18 08:12:36.887,hachem.betrouni@g.enp.edu.dz,Hachem,Betrouni,23,Algeria,Algeria,Male,Master's,Working in industry,...,Extensive experience for 4+ years,5,9,3,https://drive.google.com/open?id=1F5EgkFnvL8lw...,Haïfa Ben Messaoud/Instadeep,haifa.b.messaoud@gmail.com,Yasser Salah Eddine Bouchareb/Google,ysd.bouchareb@gmail.com,6.8
9,2024-01-16 18:38:10.348,ousseynou.mbaye@uadb.edu.sn,Ousseynou,MBAYE,38,Senegal,Senegal,Male,PhD/MD degree,PhD,...,Extensive experience for 4+ years,7,9,1,https://drive.google.com/open?id=1Yy4_dK5-Xj7x...,mouhamadou Lamine Ba,mouhamadoulamine.ba@esp.sn,Babacar Mbaye,babacar.mbaye@uadb.edu.sn,6.67
35,2024-01-18 19:46:51.687,andrewayiko40@gmail.com,Ayiko,Andrew,27,Uganda,Uganda,Male,Bachelor's degree,Undergraduate,...,Have taken courses and am studying it,9,8,3,https://drive.google.com/open?id=1wEc_xKMQQtJ4...,Mr. Simon Kloker,skloker@ndejjeuniversity.ac.ug,Mr. Kateete Twaha,tkateete@ndejjeuniversity.ac.ug,6.6
18,2024-01-17 12:26:31.608,danielle.kapsa@aims-cameroon.org,Danielle Blanche,KAPSA SELAMBI,27,Cameroon,Cameroon,Female,Master's,Working in industry,...,Have taken several courses,6,8,8,https://drive.google.com/open?id=1LVe3U1fp-QLn...,MIREILLE FANGUENG/LEIBNIZ UNIVERSITY OF HANNOVER,Mireille.Fangueng@ikg.uni-hannover.de,ARIANE MELI,a.meli-chrisko.1@research.gla.ac.uk,6.53
